<a href="https://colab.research.google.com/github/AnhelinaKavaliova/task4/blob/main/PySpark_Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=ea7109d66abea0469e632cff13e5a6cd814323aac41dc4563851a4967dfb7a94
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


#PySpark_Basics

import libraries(I provide all libs that I need when make this tasks, if you need some external import them here)

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col
from pyspark.sql.functions import max, avg, min
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, desc, col
from pyspark.sql.functions import when

create local SparkSession

In [ ]:
spark = SparkSession.builder.master("local[*]").appName("PySpark_Basic").getOrCreate()

In [ ]:
spark

read csv with inferschema

In [ ]:
%%time
df = spark.read.csv("./PySpark_Basics/ds_salaries.csv", inferSchema=True, header=True)

CPU times: user 55 ms, sys: 9.54 ms, total: 64.5 ms
Wall time: 9.56 s


read csv one more time with the same code and you will see that it almostly don't take time, because info already in SparkSession and it will not read nothing
from this file

In [ ]:
%%time
df_try = spark.read.csv("./PySpark_Basics/ds_salaries.csv", inferSchema=True, header=True)

CPU times: user 5.65 ms, sys: 2.61 ms, total: 8.26 ms
Wall time: 701 ms


write schema of scv on screen

In [ ]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- work_year: integer (nullable = true)
 |-- experience_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: integer (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remote_ratio: integer (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: string (nullable = true)



create schema of this scv

In [ ]:
schema=StructType([
    StructField("_с0", IntegerType(), True),
    StructField("work_year", IntegerType(), True),
    StructField("experience_level", StringType(), True),
    StructField("employment_type", StringType(), True),
    StructField("job_title", StringType(), True),
    StructField("salary", IntegerType(), True),
    StructField("salary_currency", StringType(), True),
    StructField("salary_in_usd", IntegerType(), True),
    StructField("employee_residence", StringType(), True),
    StructField("remote_ratio", IntegerType(), True),
    StructField("company_location", StringType(), True),
    StructField("company_siz", StringType(), True)
    ])

restart kernel without cleaning output and after restarting you need to initialize SparkSession, after initialize start execute only cells from cell with schema=
=StructType....
To restart kernel click Kernel, Restart.

read ds_salaries with predefined schema and compare results from this cell and cell with inferSchema

In [ ]:
%%time
df_schema = spark.read.csv("./PySpark_Basics/ds_salaries.csv", schema=schema, header=True)

CPU times: user 26.4 ms, sys: 2.62 ms, total: 29 ms
Wall time: 2.81 s


this happens because read operation is lazy(transformation), but if you use inferschema it start to be action that will create Spark Job, because Spark need to loop throw all file to check datatypes for all columns and this can harm to your code(if we compare to parquet, it will also go to check data types, but parquet provide meta information, so Spark will not go throw all file, he will just read meta information, but csv don't provide such meta information). Also header make Spark to create one more Spark Job to check first line
to define name of columns and remember to skeep it when reading. Actual reading start when you will use first action. More about Spark Jobs you will see in next topic

write schema of scv on screen one more time and compare with previous

In [ ]:
df_schema.printSchema()

root
 |-- _с0: integer (nullable = true)
 |-- work_year: integer (nullable = true)
 |-- experience_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: integer (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remote_ratio: integer (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_siz: string (nullable = true)



now continue to work with one of the dataframes that you create

print data in dataframe using df.show

In [ ]:
df_schema.show()

+---+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+-----------+
|_с0|work_year|experience_level|employment_type|           job_title|  salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_siz|
+---+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+-----------+
|  0|     2020|              MI|             FT|      Data Scientist|   70000|            EUR|        79833|                DE|           0|              DE|          L|
|  1|     2020|              SE|             FT|Machine Learning ...|  260000|            USD|       260000|                JP|           0|              JP|          S|
|  2|     2020|              SE|             FT|   Big Data Engineer|   85000|            GBP|       109024|                GB|          50|          

print data in dataframe using display(df.toPandas())

In [ ]:
df_schema.toPandas()

,_с0,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_siz
0,0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
3,3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L
...,...,...,...,...,...,...,...,...,...,...,...,...
602,602,2022,SE,FT,Data Engineer,154000,USD,154000,US,100,US,M
603,603,2022,SE,FT,Data Engineer,126000,USD,126000,US,100,US,M
604,604,2022,SE,FT,Data Analyst,129000,USD,129000,US,0,US,M
605,605,2022,SE,FT,Data Analyst,150000,USD,150000,US,100,US,M


create df_job_title that consists from all job_titles without duplicates

In [ ]:
df_job_title = df_schema.select("job_title").distinct()


In [ ]:
df_schema.select("job_title").count()

607

In [ ]:
df_job_title.count()

50

print all rows from df_job_titles without truncating jobs

In [ ]:
df_job_title.show(truncate=False)

+----------------------------------------+
|job_title                               |
+----------------------------------------+
|3D Computer Vision Researcher           |
|Lead Data Engineer                      |
|Head of Machine Learning                |
|Data Specialist                         |
|Data Analytics Lead                     |
|Machine Learning Scientist              |
|Lead Data Analyst                       |
|Data Engineering Manager                |
|Staff Data Scientist                    |
|ETL Developer                           |
|Director of Data Engineering            |
|Product Data Analyst                    |
|Principal Data Scientist                |
|AI Scientist                            |
|Director of Data Science                |
|Machine Learning Engineer               |
|Lead Data Scientist                     |
|Machine Learning Infrastructure Engineer|
|Data Science Engineer                   |
|Machine Learning Manager                |
+----------

create  df_analytic that will consists from max, avg, min USD salaries for all job_titles using groupBy. name of fields is avg_salary, min_salary, max_salary

In [ ]:
df_analytic = df_schema.groupBy("job_title") \
  .agg(max("salary_in_usd").alias("max_salary"),
       min("salary_in_usd").alias("min_salary"),
       avg("salary_in_usd").alias("avg_salary"))


print all rows from df_analytic without trancating jobs

In [ ]:
df_analytic.show(truncate=False)

+----------------------------------------+----------+----------+------------------+
|job_title                               |max_salary|min_salary|avg_salary        |
+----------------------------------------+----------+----------+------------------+
|3D Computer Vision Researcher           |5409      |5409      |5409.0            |
|Lead Data Engineer                      |276000    |56000     |139724.5          |
|Head of Machine Learning                |79039     |79039     |79039.0           |
|Data Specialist                         |165000    |165000    |165000.0          |
|Data Analytics Lead                     |405000    |405000    |405000.0          |
|Machine Learning Scientist              |260000    |12000     |158412.5          |
|Lead Data Analyst                       |170000    |19609     |92203.0           |
|Data Engineering Manager                |174000    |59303     |123227.2          |
|Staff Data Scientist                    |105000    |105000    |105000.0    

now you need to add in df_analytic column row_id, that will show order of all job_titles depending on avg salary. they should be descending

In [ ]:
windowSpec = Window.orderBy(desc("avg_salary"))
df_analytic = df_analytic.withColumn("row_id", row_number().over(windowSpec))

print all data from df_analytic

In [ ]:
df_analytic.show(truncate=False)

+----------------------------------+----------+----------+------------------+------+
|job_title                         |max_salary|min_salary|avg_salary        |row_id|
+----------------------------------+----------+----------+------------------+------+
|Data Analytics Lead               |405000    |405000    |405000.0          |1     |
|Principal Data Engineer           |600000    |185000    |328333.3333333333 |2     |
|Financial Data Analyst            |450000    |100000    |275000.0          |3     |
|Principal Data Scientist          |416000    |148261    |215242.42857142858|4     |
|Director of Data Science          |325000    |130026    |195074.0          |5     |
|Data Architect                    |266400    |90700     |177873.9090909091 |6     |
|Applied Data Scientist            |380000    |54238     |175655.0          |7     |
|Analytics Engineer                |205300    |135000    |175000.0          |8     |
|Data Specialist                   |165000    |165000    |165000.

it isn't beautifull, so we need to put now row_id on first place in df_analytic

In [ ]:
df_analytic = df_analytic.select("row_id", "job_title", "avg_salary", "min_salary", "max_salary")

print df_analytic now

In [ ]:
df_analytic.show(truncate=False)

+------+----------------------------------+------------------+----------+----------+
|row_id|job_title                         |avg_salary        |min_salary|max_salary|
+------+----------------------------------+------------------+----------+----------+
|1     |Data Analytics Lead               |405000.0          |405000    |405000    |
|2     |Principal Data Engineer           |328333.3333333333 |185000    |600000    |
|3     |Financial Data Analyst            |275000.0          |100000    |450000    |
|4     |Principal Data Scientist          |215242.42857142858|148261    |416000    |
|5     |Director of Data Science          |195074.0          |130026    |325000    |
|6     |Data Architect                    |177873.9090909091 |90700     |266400    |
|7     |Applied Data Scientist            |175655.0          |54238     |380000    |
|8     |Analytics Engineer                |175000.0          |135000    |205300    |
|9     |Data Specialist                   |165000.0          |165

here you need to create df_exp_lvl with the biggest usd_salary(biggest_salary) for each experience_level(you need to save all fields like in entire dataframe)

In [ ]:
windowSpec = Window.partitionBy("experience_level").orderBy(desc("salary_in_usd"))
df_exp_lvl = df_schema.withColumn("row_num", row_number().over(windowSpec))
df_exp_lvl = df_exp_lvl.filter(col("row_num") == 1).drop("row_num")

print here df_exp_lvl

In [ ]:
df_exp_lvl.show(truncate=False)

+---+---------+----------------+---------------+-------------------------+------+---------------+-------------+------------------+------------+----------------+-----------+
|_с0|work_year|experience_level|employment_type|job_title                |salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_siz|
+---+---------+----------------+---------------+-------------------------+------+---------------+-------------+------------------+------------+----------------+-----------+
|37 |2020     |EN              |FT             |Machine Learning Engineer|250000|USD            |250000       |US                |50          |US              |L          |
|252|2021     |EX              |FT             |Principal Data Engineer  |600000|USD            |600000       |US                |100         |US              |L          |
|33 |2020     |MI              |FT             |Research Scientist       |450000|USD            |450000       |US                |0    

create df_best that consists from rows where salary of guy same as biggest salary for other people in his exp_lvl and choose only columns: id, experience_level, biggest_salary, employee_residence

In [ ]:
df_exp_lvl = df_exp_lvl.withColumnRenamed("salary_in_usd", "biggest_salary").withColumnRenamed("experience_level", "exp_lvl")
df_best = df_schema.join(df_exp_lvl.select("exp_lvl", "biggest_salary"), df_schema.experience_level == df_exp_lvl.exp_lvl)
df_best = df_best.filter(df_best["salary_in_usd"] == df_exp_lvl["biggest_salary"])

In [ ]:
df_best = df_best.select(col("_с0").alias("id"), "experience_level", "biggest_salary", "employee_residence")

print df_best

In [ ]:
df_best.show()

+---+----------------+--------------+------------------+
| id|experience_level|biggest_salary|employee_residence|
+---+----------------+--------------+------------------+
| 33|              MI|        450000|                US|
| 37|              EN|        250000|                US|
| 63|              SE|        412000|                US|
| 97|              MI|        450000|                US|
|252|              EX|        600000|                US|
+---+----------------+--------------+------------------+



drop duplicates if exist by experience_level

In [ ]:
df_best = df_best.dropDuplicates(["experience_level"])

print df_best

In [ ]:
df_best.show()

+---+----------------+--------------+------------------+
| id|experience_level|biggest_salary|employee_residence|
+---+----------------+--------------+------------------+
| 37|              EN|        250000|                US|
|252|              EX|        600000|                US|
| 33|              MI|        450000|                US|
| 63|              SE|        412000|                US|
+---+----------------+--------------+------------------+



create df_new_best from df_best without id, and make the next: when exp_level = MI we want middle, when SE we want senior, else Null

In [ ]:
df_new_best = df_best.drop("id")
df_new_best = df_new_best.withColumn("high_exp_lvl", when(df_new_best.experience_level == "MI", "middle")
                                                    .when(df_new_best.experience_level == "SE", "senior")
                                                    .otherwise("Null"))

print df_new_best

In [ ]:
df_new_best.show()

+----------------+--------------+------------------+------------+
|experience_level|biggest_salary|employee_residence|high_exp_lvl|
+----------------+--------------+------------------+------------+
|              EN|        250000|                US|        Null|
|              EX|        600000|                US|        Null|
|              MI|        450000|                US|      middle|
|              SE|        412000|                US|      senior|
+----------------+--------------+------------------+------------+



write df_new_best like 1.csv and load then it to df_final

In [ ]:
df_new_best.write.csv("/PySpark_Basics/1.csv", header=True, mode='overwrite')
df_final = spark.read.csv("/PySpark_Basics/1.csv", header=True, inferSchema=True)

print df_final

In [ ]:
df_final.show()

+----------------+--------------+------------------+------------+
|experience_level|biggest_salary|employee_residence|high_exp_lvl|
+----------------+--------------+------------------+------------+
|              EN|        250000|                US|        Null|
|              EX|        600000|                US|        Null|
|              MI|        450000|                US|      middle|
|              SE|        412000|                US|      senior|
+----------------+--------------+------------------+------------+



filter df_final to delete experience_level where it Null, then join this table by biggest_salary(salary_in_usd) and employee_residence with entire df

In [ ]:
df_final = df_final.filter(col("high_exp_lvl") != "Null")

In [ ]:
df_final.show()

+----------------+--------------+------------------+------------+
|experience_level|biggest_salary|employee_residence|high_exp_lvl|
+----------------+--------------+------------------+------------+
|              MI|        450000|                US|      middle|
|              SE|        412000|                US|      senior|
+----------------+--------------+------------------+------------+



In [ ]:
df_final_try = df_final.join(df_schema, (df_schema.salary_in_usd == df_final.biggest_salary) &
                                    (df_schema.employee_residence == df_final.employee_residence), 'inner')

print df_final

In [ ]:
df_final_try.show()

+----------------+--------------+------------------+------------+---+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+-----------+
|experience_level|biggest_salary|employee_residence|high_exp_lvl|_с0|work_year|experience_level|employment_type|           job_title|salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_siz|
+----------------+--------------+------------------+------------+---+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+-----------+
|              MI|        450000|                US|      middle| 33|     2020|              MI|             FT|  Research Scientist|450000|            USD|       450000|                US|           0|              US|          M|
|              SE|        412000|                US|      senior| 63|   

last task is to save in variable and then print this variable of the biggest salary_in_usd from df_final

In [ ]:
biggest_salary = df_final.select(max("biggest_salary")).collect()[0][0]
print(biggest_salary)

450000


It is the end of PySpark basics. In other lessons you will learn optimizations technics and how to make distributed system